# Histogram obrazu. Wyrównywanie histogramu.

## Cel ćwiczenia

- Zapoznanie z pojęciem histogramu obrazu (w odcieniach szarości i kolorze).
- Zapoznanie z metodami modyfikacji histogramu (rozciąganie, wyrównywanie, dopasowywanie).

## Histogram

- Histogramem obrazu nazywamy wykres słupkowy zdefiniowany następującymi zależnościami:<br>
\begin{equation}
h(i) = \sum_{x=0}^{N-1} \sum_{y=0}^{M-1} p(i,(x,y))
\end{equation}<br>
gdzie:<br>
\begin{equation}
p(i) =  \left\{
  \begin{array}{l l}
    1 & \quad \text{gdy} f(x,y) = i\\
    0 & \quad \text{gdy} f(x,y) \ne i
  \end{array} \right.
\end{equation}

- Inaczej mówiąc, histogram zawiera informacje na temat tego ile pikseli o danym poziomie jasności występuje na obrazie (w przypadku obrazu w odcieniach szarości). Określa się to także rozkładem empirycznym cechy.

- Często wykorzystuje się tzw. znormalizowaną postać histogramu  – wszystkie wartości $h(i)$ są dzielone przez liczbę pikseli na obrazie.
Otrzymana w ten sposób wielkość to gęstość prawdopodobieństwa wystąpienia na obrazie pikseli o odcieniu $i$.

- Histogram można zdefiniować również dla obrazów kolorowych.
Otrzymujemy wtedy 3 histogramy – po jednym dla danej składowej: R,G,B (lub HSV, YCbCr, itp.) lub histogram trójwymiarowy.

- Histogram jest bardzo użyteczny w przetwarzaniu i analizie obrazów.
Wykorzystywany jest przy binaryzacji (szerzej na jednym z kolejnych laboratoriów) oraz do oceny jakości (dynamiki, kontrastu) obrazu.
W idealnym przypadku wszystkie poziomy jasności w obrazie powinny być wykorzystane (i to najlepiej w miarę jednolicie)  – obrazowo mówiąc histogram powinien rozciągać się od 0  – 255 (obraz w skali szarości).

- W przypadku gdy  wykorzystujemy jedynie fragment dostępnego zakresu (wąski histogram)  lub histogram nie jest jednolity (występują dominujące grupy pikseli) obraz ma dość słaby kontrast.
Cechę tę można poprawić stosując tzw. rozciąganie albo wyrównywanie histogramu (ang. *histogram equalization*).</div>

## Histogram dla obrazów w odcieniach szarości

1. Zaimportuj potrzebne biblioteki: *OpenCV*, *pyplot* z *matplotlib* i *numpy*.
        import cv2
        from matplotlib import pyplot as plt
        import numpy as np
2. Wczytaj obrazy *lenaX.bmp* w skali szarości. *X* jest numerem wczytywanego obrazu (1 - 4).
        I = cv2.imread('lenaX.bmp', cv2.IMREAD_GRAYSCALE)
3. Oblicz histogram wczytanego obrazu wykorzystując funkcję `cv2.calcHist`.
    - Pierwszym argumentem jest obraz, dla którego obliczony zostanie histogram.
    Należy go przekazać w nawiasie kwadratowym.
    - Drugim argumentem jest numer kanału, dla którego ma zostać obliczony histogram.
    Również powinien być przekazany w nawiasie kwadratowym.
    - Trzeci argument oznacza maskę, czyli obszar, dla którego ma zostać wyznaczony histogram.
    Aby obliczyć dla całego obrazu należy przekazać *None*.
    - Czwartym argumentem jest rozmiar histogramu (liczba przedziałów).
    Argument powinien być w nawiasie kwadratowym. Dla pełnej skali należy przekazać wartość *256*.
    - Ostatnim argumentem jest zakres wartości. Dla obrazów typu *uint8* powinien on wynosić *[0, 256]*.
    - Funkcja zwraca obliczony histogram.
4. Wyświetl wczytane obrazy i ich histogramy w jednym oknie. Użyj `plt.subplot()` w celu stworzenia siatki wykresów.
        figLena, axsLena = plt.subplots(2, 4)
Rozmiar utworzonego okna można zmienić wykorzystując instrukcję (uwaga w calach -  1 cal to 2.54cm):
        figLena.set_size_inches(20, 10)
Przykładowe wyświetlenie obrazu:
        axsLena[0, 0].imshow(I1, 'gray', vmin=0, vmax=256)
        axsLena[0, 0].axis('off')
Przykładowe wyświetlenie histogramu:
        axsLena[1, 0].plot(H1)
        axsLena[1, 0].grid()
5. Przeanalizuj (dokładnie) związek histogramu z jasnością i ostrością obrazu (tu rozumianą jako subiektywne odczucie).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'

images = []

fileName = 'lena1.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

fileName = 'lena2.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

fileName = 'lena3.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

fileName = 'lena4.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

In [ ]:
def show_hists(images):
    figLena, axesLena = plt.subplots(4, 2)
    figLena.set_size_inches(10, 20)

    for num, img in enumerate(images):
        histogram = cv2.calcHist([img], [0], None, [256], [0, 256])
        axesLena[num, 0].plot(histogram)
        axesLena[num, 0].grid()
        axesLena[num, 1].imshow(img, 'gray', vmin=0, vmax=256)
        axesLena[num, 1].axis('off')
    plt.show()

show_hists(images)

## Rozciąganie histogramu

Najprostszą metodą poprawienia jakości obrazu jest tzw. rozciągnięcie histogramu.
Polega na przeskalowaniu wartości pikseli w obrazie tak, aby wykorzystać cały dostępny zakres [0-255] (oczywiście w przypadku obrazów w odcieniach szarości w reprezentacji 8-bitowej).

1. Wczytaj obraz *hist1.bmp* w skali szarości.
Oblicz i wyświetl histogram rozpatrywanego obrazu (na wspólnym rysunku z obrazem).
Zwróć uwagę na ilość widocznych szczegółów.
2. Rozciągnij histogram obrazu. W tym celu można wykorzystać funkcję `cv2.normalize`.
    - Pierwszym argumentem funkcji jest obraz poddawany operacji.
    - Drugim argumentem jest tablica do której zostanie wpisany wynik.
    Należy ją najpierw zainicjalizować.
    Najlepiej zrobić to funkcją `np.zeros`, której pierwszym argumentem jest rozmiar obrazu (`I.shape`), a drugim typ danych (`uint8`).
    Można również przekazać `None`, a wynik przypisać do nowej zmiennej.
    - Trzecim argumentem jest minimalna wartość po normalizacji.
    - Czwartym argumentem jest wartość maksymalna po normalizacji.
    - Ostatnim argumentem jest typ wykorzystanej normy (uogólnienie pojęcia długości wektora).
    Należy wykorzystać normę `cv2.NORM_MINMAX`.
3. Wyświetl obraz oryginalny, po wykonanej operacji oraz ich histogramy.
4. Czy ilość "widocznych" szczegółów uległa zmianie?

In [ ]:
images_hist = []

fileName = 'hist1.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images_hist.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

fileName = 'hist2.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images_hist.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

fileName = 'hist3.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images_hist.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

fileName = 'hist4.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
images_hist.append(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE))

In [ ]:
# show_hists(images_hist)

In [ ]:
def normalize_hists(images):
    normalized_images = []
    for img in images:
        normal_img = cv2.normalize(img, None, 0, 256, cv2.NORM_MINMAX)
        normalized_images.append(normal_img)
    return normalized_images
    plt.show()

after_normalization = normalize_hists(images_hist)
# show_hists(after_normalization)

In [ ]:
def compare(images_1, images_2):
    fig, axes = plt.subplots(4, len(images_1))
    fig.set_size_inches(20, 10)
    for num, (img_1, img_2) in enumerate(zip(images_1, images_2)):
        his_1 = cv2.calcHist([img_1], [0], None, [256], [0, 256])
        axes[num, 0].plot(his_1)
        axes[num, 0].grid()
        axes[num, 1].imshow(img_1, 'gray', vmin=0, vmax=256)
        axes[num, 1].axis('off')
        axes[num, 2].imshow(img_2, 'gray', vmin=0, vmax=256)
        axes[num, 2].axis('off')
        his_2 = cv2.calcHist([img_2], [0], None, [256], [0, 256])
        axes[num, 3].plot(his_2)
        axes[num, 3].grid()
    plt.show()

compare(images_hist, after_normalization)

## Wyrównywanie histogramu

<div style="text-align: justify">
Bardziej zaawansowaną metodą jest tzw. wyrównywanie histogramu (ang. *histogram equalization – HE*).
Idea jest następująca: z punktu widzenia lepszego wykorzystania dostępnych poziomów jasności pożądane jest rozciągnięcie "szczytów" histogramu, a~skompresowanie "dolin" tak, aby taka sama liczba pikseli reprezentowana była przez każdy z dostępnych poziomów jasności (a przynjamniej przybliżona).
Warto zwrócić uwagę, że takie przekształcenie powoduje częściową utratę informacji o szczegółach w obszarach "dolin".
Inaczej mówiąc, dążymy do sytuacji, aby histogram był względnie jednostajny.
Operacją, która pozwala wykonać wyrównywanie histogramu, jest przekształcenie LUT z funkcją przejścia w postaci histogramu skumulowanego danego obrazu.</div><br>

<div style="text-align: justify">
Histogram skumulowany to funkcja obliczona na podstawie histogramu.
Jej pierwszy element to liczba pikseli o odcieniu $0$.
Kolejne wartości to liczba pikseli o odcieniach od $0$ do $n$.</div>

\begin{equation}
C(n) = \sum_{i=0}^{n} h(i)
\end{equation}

<div style="text-align: justify">
Jeżeli histogram jest w postaci znormalizowanej (gęstość rozkładu prawdopodobieństwa) to histogram skumulowany stanowi dystrybuantę rozkładu prawdopodobieństwa.</div><br>

1. Wyznacz histogram skumulowany dla obrazu *hist1.bmp*.
W tym celu wykorzystaj metodę `cumsum` dla histogramu wczytanego obrazu.
Nie przyjmuje ona żadnych argumentów, a zwraca skumulowane wartości tablicy, dla której została użyta.
Histogram należy wyliczyć dla **obrazka wejściowego**, a nie dla wyniku rozciągania.
2. Histogram skumulowany wyświetl razem z histogramem zwykłym na jednym wykresie (nie obok siebie).
Na potrzeby wyświetlenia przeskaluj histogram skumulowany tak, by miał taką samą wartość maksymalną jak zwykły histogram.
W tym celu wykorzystaj metodę `max`.
3. Wyświetlenie kilku linii na jednym wykresie może być zrealizowane w następujący sposób:
        figHistCum, axsHistCum = plt.subplots()

        axsHistCum.plot(HHist)
        axsHistCum.plot(CHistNorm)
        axsHistCum.grid()
4. Teraz zaimplementuj klasyczny algorytm wyrównywania histogramu.
Wykorzystać należy obliczony histogram skumulowany.
Należy go przeskalować w taki sposób aby na jego podstawie zrealizować przekształcenie LUT, czyli do zakresu 0 - 255.

>Uwaga. Opisany algorytm wyrównywania histogramu jest wersją uproszczoną.
>W wersji pełnej należy podczas skalowania tablicy przekodowań LUT pominąć elementy równe *0*.
>
>W tym celu należy wykorzystać funkcje `np.ma.masked_equal` i `np.ma.filled`.
>Pierwsza służy do ukrywania elementów tablicy, natomiast druga zamienia ukryte elementy na podaną wartość.
>W tym przypadku elementem ukrywanym i wpisywaną wartością byłoby *0*.

5. Na kolejnym rysunku wyświetl obrazek po przekształceniu, jego histogram oraz histogram skumulowany.
Co szczególnego można powiedzieć o jego histogramie i histogramie skumulowanym?
6. W bibliotece *OpenCV* dostępna jest funkcja wykonująca wyrównywanie histogramu `cv2.equalizeHist`.
Jej argumentem jest obraz, którego histogram zostanie wyrównany. Zwraca natomiast obraz wynikowy.
Na kolejnym rysunku wyświetl wynik funkcji, jego histogram oraz histogram skumulowany.
7. W wykorzystywanej bibliotece zaimplementowana jest również metoda adaptacyjnego wyrównywania histogramu algorytmem CLAHE (ang. *Contrast Limited Adaptive Histogram Equalization*}.
   Kilka słów wyjaśnienia.
   Wadą poznanej metody HE jest jej "globalność" rozumiana jako nieuwzględnianie lokalnych właściwości obrazu.
   Dlatego też powstała metoda adaptacyjnego wyrównywania histogramu (AHE).
   Jest ona spotykana w dwóch wariantach:
   - dla każdego piksela niezależnie, w pewnym jego otoczeniu, wyznaczany jest histogram i przeprowadzane wyrównywanie.
     Jak nietrudno się domyślić rozwiązanie jest dość kosztowne obliczeniowo.
   - obraz wejściowy dzielony jest na nienachodzące na siebie prostokątne okna.
     W każdym z okien obliczany jest histogram i przeprowadzane jest wyrównywanie.
     W celu eliminacji błędów na granicy okien, stosuje się interpolację.

   Metoda AHE ma jednak tą wadę, że w obszarach jednorodnych wzmacnia szum.
   Dlatego też zaproponowano rozwiązanie CLAHE, które zakłada ograniczenie kontrastu (CL).
   W metodzie definiuje się maksymalną wartość danego przedziału histogramu (próg ograniczenia kontrastu).
   Piksele, które przekraczają próg są następnie równomiernie rozdzielane pomiędzy poszczególne przedziały.
   Bardziej szczegółowy opis obu metod dostępny jest na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization).

8.W celu użycia algorytmu należy wywołać funkcję `cv2.createCLAHE`.
    - Pierwszym parametrem jest próg ograniczenia kontrastu.
    - Drugi parametr definiuje na ile prostokątów zostanie podzielony obraz w rzęch i kolumnach.
    - Zwracany jest zainicjalizowany *smart pointer* do klasy `cv::CLAHE`.
9. Aby wykonać wyrównywanie należy użyć metody `apply`.
Jej argumentem jest obraz wejściowy. Zwracany jest obraz o zmodyfikowanym histogramie.
10. Przetestuj różne parametry algorytmu CLAHE.
11. W kolejnym etapie należy przetestować operacje (rozciąganie, wyrównywanie (HE) i adaptacyjne wyrównywanie CLAHE)  na histogramie dla obrazów rzeczywistych. *hist2.bmp*, *hist3.bmp*, *hist4.jpg*.
W jednym oknie wyświetl: obraz oryginalny, rozciąganie, wyrównywanie HE oraz wyrównywanie CLAHE.

In [ ]:
def compare_hists(images):
    fig, axes = plt.subplots(len(images), 8)
    fig.set_size_inches(60, 40)

    def show(row, col, img):
        hist = cv2.calcHist([img], [0], None, [256], [0, 256])
        hist_cumsum = hist.cumsum()
        hist_cumsum_normalized = hist_cumsum/max(hist_cumsum) * max(hist)
        axes[row, col].imshow(img, 'gray', vmin=0, vmax=255)
        axes[row, col+1].plot(hist_cumsum_normalized, label='Histogram skumulowany')
        axes[row, col+1].plot(hist, label='Histogram')
        axes[row, col+1].legend()
        axes[row, col+1].grid()

    for num, img in enumerate(images):
        show(num, 0, img)
        axes[num, 0].set_title("Input photo")

        hist = cv2.calcHist([img], [0], None, [256], [0, 256])
        hist_cumsum = hist.cumsum()
        hist_cumsum_no_zero = np.ma.masked_equal(hist_cumsum, 0)
        hist_cumsum_no_zero = hist_cumsum_no_zero/max(hist_cumsum) * 255
        hist_cumsum = np.ma.filled(hist_cumsum_no_zero, 0).astype('uint8')

        img_equal_lut = cv2.LUT(img, hist_cumsum)
        show(num, 2, img_equal_lut)
        axes[num, 2].set_title("Equalized photo (LUT version)")

        img_equal_cv2 = cv2.equalizeHist(img)
        show(num, 4, img_equal_cv2)
        axes[num, 4].set_title("Equalized photo (cv2 function)")

        pointer = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img_clahe = pointer.apply(img)
        show(num, 6, img_clahe)
        axes[num, 6].set_title("Equalized by CLAHE algorithm")

    plt.show()

compare_hists(images_hist)

In [ ]:
def show_transforms(images):
    fig, axes = plt.subplots(len(images), 4)
    fig.set_size_inches(80, 60)

    for num, img in enumerate(images):
        axes[num, 0].imshow(img, 'gray', vmin=0, vmax=255)
        axes[num, 0].set_title("Original photo")

        img_normal = cv2.normalize(img, None, 0, 256, cv2.NORM_MINMAX)
        axes[num, 1].imshow(img_normal, 'gray', vmin=0, vmax=255)
        axes[num, 1].set_title("Zrównoważenie histogramu")

        img_equal_cv2 = cv2.equalizeHist(img)
        axes[num, 2].imshow(img_equal_cv2, 'gray', vmin=0, vmax=255)
        axes[num, 2].set_title("Wyrównanie histogramu")

        pointer = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(32, 32))
        img_clahe = pointer.apply(img)
        axes[num, 3].imshow(img_clahe, 'gray', vmin=0, vmax=255)
        axes[num, 3].set_title("Wyrównanie histogramu CLAHE algorytm")

    plt.show()

show_transforms(images_hist)

## Histogram dla obrazów kolorowych i jego wyrównywanie

1. Wczytaj obraz *lenaRGB.bmp*.
2. Wykonaj konwersję przestrzeni barw z BGR do RGB.
3. Wyświetl wczytany obraz oraz histogram dla każdej składowej przestrzeni barw.
W tym celu można użyć drugiego argumentu wykorzystywanej funkcji (numer kanału).
4. Wykonaj wyrównywanie dla każdej składowej obrazu. Połącz otrzymane składowe w nowy obraz i wyświetl go.
Jaka jest zasadnicza wada takiego podejścia?
5. Przekształć obraz wejściowy do przestrzeni HSV (flaga `cv2.COLOR_BGR2HSV`).
Wyświetl histogramy poszczególnych składowych.
Manipulacji dokonujemy na składowej odpowiadającej za jasność, czyli V.
Wykonaj wyrównywanie histogramu dla tej składowej.
Dokonaj podmiany składowej V i wyświetl rezultat operacji.
Uprzednio przeprowadź konwersję HSV->RGB (flaga `cv2.COLOR_HSV2RGB`).
6. Na koniec wykonaj te same operacje dla obrazu *jezioro.jpg*.

In [ ]:
images_rgb = []

fileName = 'lenaRGB.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
img = cv2.imread(fileName, cv2.IMREAD_COLOR_BGR)
img_rgb = cv2.cvtColor(img, code=cv2.COLOR_BGR2RGB)
images_rgb.append(img_rgb)

fileName = 'jezioro.jpg'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)
img = cv2.imread(fileName, cv2.IMREAD_COLOR_BGR)
img_rgb = cv2.cvtColor(img, code=cv2.COLOR_BGR2RGB)
images_rgb.append(img_rgb)

In [ ]:
fig, axes = plt.subplots(len(images_rgb), 2)
fig.set_size_inches(10, 6)
colours = ["red", "green", "blue"]
channels = ["red", "green", "blue"]

def show_color_hists(image, row, axes, start):
    axes[row, start].imshow(image, "viridis", vmin=0, vmax=255)
    axes[row, start].axis("off")
    for i, colour in enumerate(colours):
        hist = cv2.calcHist([image], [i], None, [256], [0, 255])
        axes[row, start+1].plot(hist, label=f"Channel: {channels[i]}", color=colour)
    axes[row, start+1].grid()
    axes[row, start+1].legend()

for row, img in enumerate(images_rgb):
    show_color_hists(img, row, axes, 0)

plt.show()

In [ ]:
fig, axes = plt.subplots(len(images_rgb), 6)
fig.set_size_inches(25, 8)

for num, img in enumerate(images_rgb):
    show_color_hists(img, num, axes, 0)
    equal_img = img.copy()
    for i, color in enumerate(colours):
        equal_img[:, :, i] = cv2.equalizeHist(img[:, :, i])
    show_color_hists(equal_img, num, axes, 2)
    for i, color in enumerate(colours):
        pointer = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        equal_img[:, :, i] = pointer.apply(img[:, :, i], None)
    show_color_hists(equal_img, num, axes, 4)

In [ ]:
fig, axes = plt.subplots(len(images_rgb), 4)
fig.set_size_inches(20, 6)

for num, img in enumerate(images_rgb):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    channels = ["h", "v", "s"]
    show_color_hists(hsv, num, axes, 0)
    hsv_equal = hsv.copy()
    hsv_equal[:, :, 2] = cv2.equalizeHist(hsv_equal[:, :, 2])
    hsv_equal = cv2.cvtColor(hsv_equal, cv2.COLOR_HSV2RGB)
    channels = ["red", "green", "blue"]
    show_color_hists(hsv_equal, num, axes, 2)